In [3]:
from poitousprint import Toflit

client = Toflit()

flows = client.get_flows(year=1789, customs_region="La Rochelle", best_guess_region_prodxpart='1')
len(flows)

1063

In [2]:
africa_products = {
    "Exports": {},
    "Imports": {}
}
colonies_products = {
    "Exports": {},
    "Imports": {}
}

for f in flows :
    
    product_viz = ''
    product = f['product_revolutionempire']
    value = f['value'].split('.')[0] if f['value'].split('.')[0] != '' else 0
    flow_type = f['export_import']
    """
    if (f['product_simplification'] == 'guinée'):
        print(f['product_revolutionempire'], f['value'])
    """
    if (f['partner_grouping'] == 'Afrique'):
        africa_products[flow_type][product] = int(africa_products[flow_type][product]) + int(value) if f['product_simplification'] in africa_products[flow_type] else int(value)
    elif (f['partner_grouping'] == 'Amériques' or f['partner_grouping'] == 'Asie'):
        colonies_products[flow_type][product] = int(colonies_products[flow_type][product]) + int(value) if product in colonies_products[flow_type] else int(value)
    
    if product in ['Café', 'Sucre', 'Indigo', 'Coton non transformé']:
        product_viz = "produit colonial ('Café', 'Sucre', 'Indigo', 'Coton non transformé')"
    elif (product == 'Sel'):
        product_viz = 'sel'
    elif (product == 'Eaux-de-vie et liqueurs' or product == 'Vins divers'):
        product_viz = 'eau-de-vie et vins divers'
    else :
        product_viz = 'autres produits'
    f['product_viz'] = product_viz
    
    partner_viz = ''
    if (f['partner_simplification'] == 'Monde hors colonies'):
        partner_viz = 'Indéterminé (supposé réexportations vers Europe)'
    elif (f['partner_grouping'] == 'France'):
        partner_viz = 'Ports francs et petites îles'
    elif (f['partner_grouping'] in ['Flandre et autres états de l\'Empereur', 'Nord', 'Flandres', 'Hollande', 'Espagne', 'Portugal', 'Allemagne']):
        partner_viz = 'Europe'
    elif (f['partner_grouping'] == 'Angleterre'):
        partner_viz = 'Grande Bretagne'
    elif (f['partner_grouping'] == 'Afrique'):
        partner_viz = 'Afrique'
    elif (f['partner_grouping'] == 'Amériques' or f['partner_grouping'] == 'Asie'):
        partner_viz = 'Colonies (Saint-Domingue, Indes, îles fr de l\'Amérique)'
    else:
        partner_viz = 'Reste du monde (USA)'
        
    f['partner_viz'] = partner_viz

africa_products['Exports'] = [{"product_simplification": product_simplification, "value": int(value)} for (product_simplification, value) in africa_products['Exports'].items()]
africa_products['Imports'] = [{"product_simplification": product_simplification, "value": int(value)} for (product_simplification, value) in africa_products['Imports'].items()]


colonies_products['Exports'] = [{"product_simplification": product_simplification, "value": int(value)} for (product_simplification, value) in colonies_products['Exports'].items()]
colonies_products['Imports'] = [{"product_simplification": product_simplification, "value": int(value)} for (product_simplification, value) in colonies_products['Imports'].items()]

In [ ]:
from vega import VegaLite
import pandas as pd

africa_exports = pd.DataFrame([f for f in africa_products['Exports']])
africa_imports = pd.DataFrame([f for f in africa_products['Imports']])

colonies_exports = pd.DataFrame([f for f in colonies_products['Exports']])
colonies_imports = pd.DataFrame([f for f in colonies_products['Imports']])

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Produits concernés par les exports vers l'Afrique (grouping)",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "value",
            "type": "quantitative", 
            "title": "valeur cumulée"
        },
        "y": {
            "type": "nominal",
            "field": "product_simplification",
            "title": "Produit product_simplification",
            "sort": "-x"
        }
    }
}, africa_exports)

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Produits concernés par les imports depuis l'Afrique (grouping)",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "value",
            "type": "quantitative", 
            "title": "valeur cumulée"
        },
        "y": {
            "type": "nominal",
            "field": "product_simplification",
            "title": "Produit product_simplification",
            "sort": "-x"
        }
    }
}, africa_imports)

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Produits concernés par les exports vers les colonies ('Asie' et 'Amériques' dans grouping)",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "value",
            "type": "quantitative", 
            "title": "valeur cumulée"
        },
        "y": {
            "type": "nominal",
            "field": "product_simplification",
            "title": "Produit product_simplification",
            "sort": "-x"
        }
    }
}, colonies_exports)

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Produits concernés par les imports depuis les colonies ('Asie' et 'Amériques' dans grouping)",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "value",
            "type": "quantitative", 
            "title": "valeur cumulée"
        },
        "y": {
            "type": "nominal",
            "field": "product_simplification",
            "title": "Produit product_simplification",
            "sort": "-x"
        }
    }
}, colonies_imports)

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Distribution générale de la classification des produits (exports)",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "value",
            "type": "quantitative", 
            "aggregate": "sum",
            "title": "valeur cumulée"
        },
        "y": {
            "type": "nominal",
            "field": "product_viz",
            "title": "Produit",
            "sort": "-x"
        }
    }
}, [{**f, "value": float(f["value"] if f["value"] != '' else 0)} for f in flows if f['export_import'] == 'Exports'])

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Distribution générale de la classification des produits (imports)",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "value",
            "type": "quantitative", 
            "aggregate": "sum",
            "title": "valeur cumulée"
        },
        "y": {
            "type": "nominal",
            "field": "product_viz",
            "title": "Produit",
            "sort": "-x"
        }
    }
}, [{**f, "value": float(f["value"] if f["value"] != '' else 0)} for f in flows if f['export_import'] == 'Imports'])

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Distribution générale de la classification des produits par rapport aux partenaires",
    "mark": {"type": "rect", "tooltip": {"content": "data"}},
    "encoding": {
        "column": {
            "field": "export_import",
            "type": "nominal"
        },
        "color": {
            "field": "value",
            "type": "quantitative", 
            "aggregate": "sum",
            "title": "valeur cumulée"
        },
        "x": {
            "type": "nominal",
            "field": "partner_viz",
            "title": "Partenaire"
        },
        "y": {
            "type": "nominal",
            "field": "product_viz", 
            "title": "Produit"
        }
    }
}, [{**f, "value": float(f["value"] if f["value"] != '' else 0)} for f in flows])

In [ ]:
import csv

def format_for_viz(f):
    flow_type = 'export' if f['export_import'] == 'Exports' else 'import'
    return {
        "flow_type": flow_type,
        "customs_office": f['customs_office'],
        "product": f["product_viz"],
        "partner": f["partner_viz"],
        "value": f["value"]
    }

with open('part_2_toflit_viz_data.csv', 'w') as csvfile:
    flows_viz = [format_for_viz(f) for f in flows]
    fieldnames = flows_viz[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for f in flows_viz:
        writer.writerow(f)

with open('exports.csv', 'w') as csvfile:
    fieldnames = flows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for f in [f for f in flows if f['export_import'] == 'Exports']:
        writer.writerow(f)
        
with open('imports.csv', 'w') as csvfile:
    fieldnames = flows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for f in [f for f in flows if f['export_import'] == 'Imports']:
        writer.writerow(f)